In [1]:
import os
import pandas as pd
import numpy
import gmaps
import csv

In [2]:
from config import g_key
from config import g_key2

In [3]:
file_to_read = pd.read_csv("Data/WeatherPy_Vacation.csv")

In [4]:
weather_vacation_df = pd.DataFrame(file_to_read)
weather_vacation_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(Past 3hrs),Snow inches(Past 3hrs)
0,Shingu,JP,2020-04-06 02:35:54,33.73,135.98,57.20,40,20,14.99,0.00,0.00
1,Rikitea,PF,2020-04-06 02:35:54,-23.12,-134.97,75.45,66,0,10.40,0.00,0.00
2,Surt,LY,2020-04-06 02:35:54,31.21,16.59,56.34,68,7,5.06,0.00,0.00
3,Teguldet,RU,2020-04-06 02:35:54,57.31,88.17,37.81,72,0,10.16,0.00,0.00
4,Nobres,BR,2020-04-06 02:35:41,-14.72,-56.33,77.49,82,100,1.28,0.33,0.00
...,...,...,...,...,...,...,...,...,...,...,...
572,Alihe,CN,2020-04-06 02:36:49,50.57,123.72,37.35,49,0,13.09,0.00,0.00
573,Powell,US,2020-04-06 02:36:49,40.16,-83.08,48.20,81,1,6.93,0.00,0.00
574,San Antonio,US,2020-04-06 02:36:50,29.42,-98.49,68.00,82,90,4.70,0.00,0.00
575,Morgan City,US,2020-04-06 02:36:50,29.70,-91.21,70.00,73,1,4.70,0.00,0.00


In [5]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_option= input("Do you want it to be raining? (yes/no) ")
snow_option= input("Do you want it to be snowing? (yes/no) ")

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 90
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [6]:
# Filter the dataset to find the cities that fit the criteria.

if rain_option == "yes": 
    rain_pref = (weather_vacation_df["Rain inches(Past 3hrs)"] > 0)
else:
    rain_pref = (weather_vacation_df["Rain inches(Past 3hrs)"] == 0)
if snow_option == "yes": 
    snow_pref = (weather_vacation_df["Snow inches(Past 3hrs)"] > 0)
else:
    snow_pref = (weather_vacation_df["Snow inches(Past 3hrs)"] == 0)
    
preferred_cities_df = weather_vacation_df.loc[(weather_vacation_df["Max Temp"] <= max_temp) & \
                                       (weather_vacation_df["Max Temp"] >= min_temp)&\
                                             (rain_pref)
                                              & \
                                              (snow_pref)
                                             ] 
                                        
preferred_cities_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(Past 3hrs),Snow inches(Past 3hrs)
0,Shingu,JP,2020-04-06 02:35:54,33.73,135.98,57.20,40,20,14.99,0.0,0.0
1,Rikitea,PF,2020-04-06 02:35:54,-23.12,-134.97,75.45,66,0,10.40,0.0,0.0
2,Surt,LY,2020-04-06 02:35:54,31.21,16.59,56.34,68,7,5.06,0.0,0.0
3,Teguldet,RU,2020-04-06 02:35:54,57.31,88.17,37.81,72,0,10.16,0.0,0.0
5,San Patricio,MX,2020-04-06 02:35:54,19.22,-104.70,82.40,74,40,8.97,0.0,0.0
6,Ancud,CL,2020-04-06 02:35:54,-41.87,-73.82,39.20,93,0,5.82,0.0,0.0
7,La Orilla,MX,2020-04-06 02:35:54,17.98,-102.23,73.76,78,97,3.98,0.0,0.0
9,Ushuaia,AR,2020-04-06 02:34:45,-54.80,-68.30,39.20,84,75,9.17,0.0,0.0
12,Kargil,IN,2020-04-06 02:35:55,34.57,76.10,25.86,24,72,2.59,0.0,0.0
13,Hithadhoo,MV,2020-04-06 02:35:55,-0.60,73.08,82.89,71,3,3.89,0.0,0.0


In [7]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [8]:
import requests
# Iterate through the DataFrame.
for index, row in preferred_cities_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    #hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    
# Grab the first hotel from the results and store the name.
    try:
        preferred_cities_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

C:\Users\lampi\anaconda3\envs\Pythondata\lib\site-packages\pandas\core\indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\lampi\anaconda3\envs\Pythondata\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
preferred_cities_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Rain inches(Past 3hrs),Snow inches(Past 3hrs),Hotel Name
0,Shingu,JP,2020-04-06 02:35:54,33.73,135.98,57.20,40,20,14.99,0.0,0.0,新宮ユーアイホテル
1,Rikitea,PF,2020-04-06 02:35:54,-23.12,-134.97,75.45,66,0,10.40,0.0,0.0,Pension Maro'i
2,Surt,LY,2020-04-06 02:35:54,31.21,16.59,56.34,68,7,5.06,0.0,0.0,City Hotel
3,Teguldet,RU,2020-04-06 02:35:54,57.31,88.17,37.81,72,0,10.16,0.0,0.0,Gostinitsa Kedr
5,San Patricio,MX,2020-04-06 02:35:54,19.22,-104.70,82.40,74,40,8.97,0.0,0.0,Hotel Cabo Blanco
...,...,...,...,...,...,...,...,...,...,...,...,...
571,Pitimbu,BR,2020-04-06 02:36:49,-7.47,-34.81,78.80,94,20,3.36,0.0,0.0,Reserva do Abiaí
572,Alihe,CN,2020-04-06 02:36:49,50.57,123.72,37.35,49,0,13.09,0.0,0.0,Alihe Forestry Hotel
573,Powell,US,2020-04-06 02:36:49,40.16,-83.08,48.20,81,1,6.93,0.0,0.0,Eco Nail Spa
574,San Antonio,US,2020-04-06 02:36:50,29.42,-98.49,68.00,82,90,4.70,0.0,0.0,Marriott Plaza San Antonio


In [16]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = preferred_cities_df[["Lat", "Lng"]]
gmaps.configure(api_key=g_key)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

In [20]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in preferred_cities_df.iterrows()]

In [22]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
hotel_df = preferred_cities_df
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [37]:
hotel_column_order = ['City','Country','Max Temp','Lat', 'Lng', 'Hotel Name']
new_hotel_df = pd.DataFrame(hotel_df[hotel_column_order])
new_hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
0,Shingu,JP,57.20,33.73,135.98,新宮ユーアイホテル
1,Rikitea,PF,75.45,-23.12,-134.97,Pension Maro'i
2,Surt,LY,56.34,31.21,16.59,City Hotel
3,Teguldet,RU,37.81,57.31,88.17,Gostinitsa Kedr
5,San Patricio,MX,82.40,19.22,-104.70,Hotel Cabo Blanco
...,...,...,...,...,...,...
571,Pitimbu,BR,78.80,-7.47,-34.81,Reserva do Abiaí
572,Alihe,CN,37.35,50.57,123.72,Alihe Forestry Hotel
573,Powell,US,48.20,40.16,-83.08,Eco Nail Spa
574,San Antonio,US,68.00,29.42,-98.49,Marriott Plaza San Antonio
